In [0]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold,StratifiedShuffleSplit
from pyspark.sql.functions import countDistinct
from sklearn.preprocessing import StandardScaler
import collections
from collections import namedtuple

from sklearn import svm
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import tree
from sklearn.naive_bayes     import MultinomialNB
from sklearn.ensemble        import RandomForestClassifier
from sklearn.linear_model    import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

import operator
import itertools
from collections import namedtuple
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, precision_recall_fscore_support

from pyspark.sql.types import StructField
from pyspark.sql.types import StructType

In [0]:
original_data = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/aggregated_data.csv')
main_cols= ['accelX(g)_mean', 'accelX(g)_std', 'accelX(g)_min', 'accelX(g)_max',
            'accelY(g)_mean', 'accelY(g)_std', 'accelY(g)_min', 'accelY(g)_max',
            'accelZ(g)_mean', 'accelZ(g)_std', 'accelZ(g)_min', 'accelZ(g)_max',
            'gyroX(rad/s)_mean', 'gyroX(rad/s)_std', 'gyroX(rad/s)_min',
            'gyroX(rad/s)_max', 'gyroY(rad/s)_mean', 'gyroY(rad/s)_std',
            'gyroY(rad/s)_min', 'gyroY(rad/s)_max', 'Speed(m/s)_min', 'Speed(m/s)_max',
            'Speed(m/s)_mean', 'Speed(m/s)_std', 
             'energy_y', 'energy_x', 'energy_z',
             'entropy_y', 'entropy_x', 'entropy_z', 'accel_mag_mean','accel_mag_std', 'accel_mag_min', 
            'accel_mag_max','gyro_mag_mean','gyro_mag_std', 'gyro_mag_min','gyro_mag_max','label']
reduced_cols=['Speed(m/s)_mean', 'Speed(m/s)_std', 'Speed(m/s)_min', 'Speed(m/s)_max',
         'accel_mag_mean','accel_mag_std', 'accel_mag_min', 'accel_mag_max','gyro_mag_mean','gyro_mag_std', 'gyro_mag_min','gyro_mag_max', 'energy_y', 'energy_x', 'energy_z','label']
sensor_data=original_data[reduced_cols]

In [0]:
potholes=sensor_data[sensor_data['label']==1]
normal_obs=sensor_data[sensor_data['label']==0] # Create a dataframe for non-pothole observations

ratio_imb=potholes.count()/normal_obs.count()
#Randomly select datapoints from normal observations to balance dataset
normal_obs_balance=normal_obs.sample(False, ratio_imb, seed=42)
balanced_sensordata=normal_obs_balance.union(potholes)

In [0]:
balanced_sensordata_pandas=balanced_sensordata.toPandas() # convert Spark dataframe to pands dataframe
dataset=balanced_sensordata_pandas.sample(frac=1, random_state=42).reset_index(drop=True)

In [0]:
# function to separate target and features

def feat_target(data):
  feat= data.drop(['label'], axis=1)
  target=data['label']
  return feat, target

In [0]:
features, target = feat_target(dataset)

In [0]:
x_train,x_test,y_train,y_test = train_test_split(features,target,test_size = 0.2)

In [0]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

Out[ 38 ]: ((551, 15), (138, 15), (551,), (138,))

In [0]:
def kfold_validation(features, target,model_config):
  # Set parameters for models we want to test
  svc_params=  {'C': [0.5, 0.7, 0.9, 1]
                , 'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
                'probability':[True]
               }
  
  log_params= {"penalty": ['l1', 'l2']
               ,'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
              }
  ModelConfig = namedtuple('ModelConfig', ['name', 'model_class', 'params'])
  model_config = [ModelConfig(name='SVC', model_class=SVC, params=svc_params),
                  ModelConfig(name='Logistic Regression', model_class=LogisticRegression, params=log_params)
  cross_val_score(model_config, features, target, cv=5,scoring='r2')
  cv_r2_scores_rf